AkHQ: http://localhost:8082  
Spark-Master http://localhost:8083  
Spark-Worker-1 http://localhost:8084

https://github.com/aehrc/pathling/tree/issue/452/lib/python#python-api-for-pathling

# Try Connect To Test FHIR Kafka

In [1]:
!python --version

Python 3.10.5


In [2]:
import os
os.environ

environ{'SHELL': '/bin/bash',
        'HOSTNAME': 'fa879c1f580b',
        'LANGUAGE': 'en_US.UTF-8',
        'SPARK_OPTS': '--driver-java-options=-Xms1024M --driver-java-options=-Xmx4096M --driver-java-options=-Dlog4j.logLevel=info',
        'NB_UID': '1000',
        'PWD': '/home/jovyan',
        'PYSPARK_VERSION': '3.2.1',
        'PYSPARK_SUBMIT_ARGS': '     --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,au.csiro.pathling:encoders:5.4.0     --repositories https://oss.sonatype.org/content/repositories/snapshots     pyspark-shell',
        'HOME': '/home/jovyan',
        'LANG': 'en_US.UTF-8',
        'NB_GID': '100',
        'XDG_CACHE_HOME': '/home/jovyan/.cache/',
        'APACHE_SPARK_VERSION': '3.2.1',
        'PYTHONPATH': '/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip:/usr/local/spark/python:',
        'HADOOP_VERSION': '3.2',
        'SHLVL': '0',
        'CONDA_DIR': '/opt/conda',
        'SPARK_HOME': '/usr/local/spark',
        'PATHLING_VERSION': '5.4.0',


In [3]:
# for debugging
#os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1 pyspark-shell"

In [4]:
appName = "Kafka, Spark and FHIR Data"
master = "spark://spark-master:7077"
#master = "local[*]"
kafka_topic = "fhir.post-gateway-kdb"

In [5]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pathling.etc import find_jar

#conf = SparkConf()
#conf.setMaster(master) \
#    .setAppName(appName) \
#    .set("spark.ui.port","0")

# get current jars
#spark_jars = conf.get("spark.jars")
# append pathling jar
#updated_spark_jars = spark_jars + ",file://" + find_jar()

#updated_spark_jars = ["/test/1.jar", "/test2.jar"]

spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .config("spark.ui.port","0") \
    .getOrCreate()

spark.sparkContext.addFile(find_jar())

In [6]:
#spark.sparkContext.getConf().get("spark.jars")
#updated_spark_jars

In [7]:
# https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html
# default for startingOffsets is "latest", but "earliest" allows rewind for missed alerts    
df = spark \
  .readStream  \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka1:19092") \
  .option("subscribe", kafka_topic) \
  .option("startingOffsets", "earliest") \
  .load()

In [8]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [9]:
query = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
          .writeStream \
          .queryName("gettable") \
          .format("memory") \
          .start()

# close connection after 30 seconds
query.awaitTermination(30)

False

In [10]:
kafka_data = spark.sql("select * from gettable")
kafka_data.show()
type(kafka_data)

+-------------------+--------------------+
|                key|               value|
+-------------------+--------------------+
|    Patient/patient|{"resourceType": ...|
|Condition/condition|{"resourceType": ...|
|Condition/condition|{"resourceType": ...|
|Procedure/procedure|{"resourceType": ...|
|Encounter/encounter|{"resourceType": ...|
|    Patient/patient|{"resourceType": ...|
|Condition/condition|{"resourceType": ...|
|Condition/condition|{"resourceType": ...|
|Procedure/procedure|{"resourceType": ...|
|Encounter/encounter|{"resourceType": ...|
+-------------------+--------------------+



pyspark.sql.dataframe.DataFrame

In [11]:
pd_df = kafka_data.toPandas()
pd_df

,key,value
0,Patient/patient,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
1,Condition/condition,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
2,Condition/condition,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
3,Procedure/procedure,"{""resourceType"": ""Bundle"",""meta"": {""security"":..."
4,Encounter/encounter,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
5,Patient/patient,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
6,Condition/condition,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
7,Condition/condition,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
8,Procedure/procedure,"{""resourceType"": ""Bundle"",""meta"": {""security"":..."
9,Encounter/encounter,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."


# Bring Pathling into the game

In [12]:
import pathling
from pathling import PathlingContext

ptl = PathlingContext.create(spark = spark)

TypeError: 'JavaPackage' object is not callable

In [ ]:
spark._jvm.au.csiro.pathling.library.PathlingContext.create()

In [ ]:
# readme-example
#n_bundles = spark.read.text('/home/jovyan/pathling/lib/python/examples/data/bundles/', wholetext=True)
#n_bundles
#type(n_bundles)

In [ ]:
kafka_data.select("value")

In [ ]:
patients = ptl.encode_bundle(kafka_data.select("value"), 'Patient')
encounter = ptl.encode_bundle(kafka_data.select("value"), 'Encounter')
condition = ptl.encode_bundle(kafka_data.select("value"), 'Condition')

In [ ]:
patients.printSchema()

In [ ]:
patients.select("id", "birthDate", "gender", "address.postalCode").toPandas()

In [ ]:
encounter.printSchema()

In [ ]:
encounter.select("id", "subject.reference", "serviceType.coding.code", "period.start", "period.end").toPandas()

In [ ]:
condition.printSchema()

In [ ]:
condition.select("id", "encounter.reference", "code.coding.code").toPandas()

In [ ]:
from pyspark.sql.functions import regexp_replace, col

pt1 = patients.select("id", "birthDate", "gender", "address.postalCode")

enc1 = encounter \
    .select( \
        col("id").alias("encounter_id"), \
        "subject.reference", \
        col("serviceType.coding.code").alias("servicetype_code"), \
        col("period.start").alias("period_start"), \
        col("period.end").alias("period_end") \
       ) \
    .withColumn("patient_id", regexp_replace("reference", "Patient/", ""))

cd1 = condition \
    .select( \
        "id", \
        "encounter.reference", \
        col("code.coding.code").alias("condition_code") \
       ) \
    .withColumn("cond_encounter_id", regexp_replace("reference", "Encounter/", ""))

In [ ]:
join1 = enc1 \
    .join(pt1, enc1.patient_id == pt1.id)
join2 = join1 \
    .select("encounter_id", "servicetype_code", "period_start", "period_end", "patient_id", "birthDate", "gender", "postalCode") \
    .join(cd1, join1.encounter_id == cd1.cond_encounter_id) \
    .select("encounter_id", "servicetype_code", "period_start", "period_end", "patient_id", "birthDate", "gender", "postalCode", "condition_code")

In [ ]:
join2.toPandas()